In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from dil_preprocess import fit_data
from dil_predict import init, predict_trees, reduce_leaky_endpoints
import pandas as pd
from http import HTTPStatus

In [ ]:
models = init(methods="all")

In [ ]:
def test_single_key():
    """Test if single headers/status-codes can be distinguished.
       Compare an empty response, with a response with only one header/status-code set and test every tree."""
    acc = {}
    # Compare headers on most normal responses?
    for header in [{"cross-origin-resource-policy": "same-origin"}, {"content-disposition": "attachment"}, {"content-type": "image/png"}, {"cross-origin-opener-policy": "same-orgin"}, {"x-content-type-options": "nosniff"}, {"x-frame-options": "deny"}]:
        # Compare headers for most common responses
        for code in ["200", "302", "404"]:
            af = pd.DataFrame([{"req_url": "a", "resp_code": code, "cookies": True, "resp_headers": header, "resp_body_info": ":,", "resp_body_hash": ""},
                               {"req_url": "a", "resp_code": code, "cookies": False, "resp_headers": {}, "resp_body_info": ":,", "resp_body_hash": ""}])
            acc = test_af(af, header, acc)
    
    for code in [str(code.value) for code in HTTPStatus] + ['999']:
        # Compare all 2XX codes with 404 and all other codes with 200
        compare_code = "200" if not code.startswith("2") else "404"
        af = pd.DataFrame([{"req_url": "a", "resp_code": code, "cookies": True, "resp_headers": {}, "resp_body_info": ":,", "resp_body_hash": ""},
                           {"req_url": "a", "resp_code": compare_code, "cookies": False, "resp_headers": {}, "resp_body_info": ":,", "resp_body_hash": ""}])
        acc = test_af(af, code, acc)
    return acc
        
def test_af(af, diff, acc):
    """Test two responses if a method can distinguish them.
       af: pandas Dataframe of size 2
       diff: the difference between the two rows in the af
       acc: acculumulator dict to save the results in"""
    # display(af)
    af = af.apply(fit_data, axis=1).to_dict()    
    af = pd.DataFrame.from_dict(af, orient="index")
    display(af)
    leaky_endpoints = predict_trees(af, log=True)
    acc[str(diff)] = leaky_endpoints
    for method in leaky_endpoints:
        display(leaky_endpoints[method])
        pass
    return acc

In [ ]:
acc = test_single_key()

In [ ]:
# Open several new bug reports based on these results?
# + test additional common combinations
for key in acc:
    print()
    print(key)
    print([method.split("/mojo/")[1] for method in acc[key].keys()])

## Distinguish two responses application

In [ ]:
import ipywidgets as widgets
from IPython.display import display, Markdown
import functools

In [ ]:
from database_connector import connect, postgresql_to_dataframe

# Connect to the database
conn = connect()
print(conn)
column_names = ["id", "url_id", "url_dict_version", "Status-Code", "body", "X-Content-Type-Options", 
                "X-Frame-Options", "Content-Type", "Content-Disposition", "Cross-Origin-Resource-Policy",
                "Cross-Origin-Opener-Policy", "Location",
                ]
# Execute the "SELECT *" query
uf = postgresql_to_dataframe(conn, "select * from leaks_urldict", column_names)
uf["url_id"] = uf["url_id"].astype("Int64")
conn.close()

In [ ]:
res = {}
for col in uf.columns:
    uniques = uf[col].unique()
    if len(uniques) < 100:
        print(col)
        print(list(uniques))
        res[col] = list(uniques)
        

In [ ]:
codes = [100, 101, 102, 103, 200, 201, 202, 203, 204, 205, 206, 207, 208, 226, 300, 301, 302, 303, 304, 305, 307, 308, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 421, 422, 423, 424, 425, 426, 428, 429, 431, 451, 500, 501, 502, 503, 504, 505, 506, 507, 508, 510, 511, 999]
bodies = ['ecocnt_html=num_frames=1,input_id=test1', 'ecocnt_html=num_frames=2', 'ecocnt_html=post_message=mes1', 'ecocnt_html=meta_refresh=0;http://172.17.0.1:8000', 'ecocnt_css=h1 {color: blue}', 'ecocnt_js=.,,.', 'ecocnt_js=var a=5;', 'ecocnt_img=width=50,height=50,type=png', 'ecocnt_vid=width=100,height=100,duration=2', 'ecocnt_audio=duration=1', 'ecocnt_pdf=a=a', 'empty']
xcto_ops = ['nosniff', 'empty']
xfo_ops = ['deny', 'empty']
ct_ops = ['text/html', 'text/css', 'application/javascript', 'video/mp4', 'audio/wav', 'image/png', 'application/pdf', 'empty']
cd_ops = ['attachment', 'empty']
coop_ops = ['same-origin', 'empty']
corp_ops = ['same-origin', 'empty']
loc_ops = ['http://172.17.0.1:8000', 'empty']

In [ ]:
browser_id, channel = "../analysis/trees/tenmin/mojo/1/event_set::embed-img.mojo".split(".mojo")[0].split("/")[-2:]

In [ ]:
# Change font size of output
display(Markdown("""<style>
td {
  font-size: 15px
}
th {
  font-size: 15px
}
</style>\n|Table|\n|--|\n|T|"""))

In [ ]:
output = widgets.Output(width="60%")

@output.capture()
def btn_eventhandler(obj, resp_dict):
    output.clear_output()
    r1 = resp_dict["one"]
    r2 = resp_dict["two"]
    responses = []
    for r in [r1, r2]:
        responses.append({"URL": "", "Status-Code": r["code"].value, "body": r["body"].value, "Content-Type": r["ct"].value, "X-Content-Type-Options": r["xcto"].value, 
                          "X-Frame-Options": r["xfo"].value, "Content-Disposition": r["cd"].value, "Location": r["loc"].value, "Cross-Origin-Opener-Policy": r["coop"].value,
                          "Cross-Origin-Resource-Policy": r["corp"].value, "cookies": ""})
    if responses[0] == responses[1]:
        display(f"Responses are the same, please change at least one attribute!")
    else:
        af = pd.DataFrame(responses)
        leaky_endpoints = predict_trees(af, log=False)
        working_channels = len(leaky_endpoints)
        #working_string = f"**{working_channels} leak channels can distinguish the two responses!**<br>"
        working_string = """**Results:**\n\n|Browser|Leak channel|Value 1|Value 2|\n|:-|:-|:-|:-|\n"""
        for method in leaky_endpoints:
            browser_id, channel = method.split(".mojo")[0].split("/")[-2:]
            browser = "Firefox 88.0" if browser_id == "1" else "Chrome 90.0"
            working_string += f"|{browser}|{channel}|{leaky_endpoints[method].iloc[0, -1]}|{leaky_endpoints[method].iloc[1, -1]}|\n"
            #working_string += f"**{browser}: {channel}, value_r1: {leaky_endpoints[method].iloc[0, -1]}, value_r2: {leaky_endpoints[method].iloc[1, -1]}**<br>"
        display(Markdown(working_string))


resp_dict = {}
table_style = {'description_width': ''}
table_layout = {'width':'auto'}


def create_header():
    resp = widgets.Button(description="", disabled=False, font_weight='bold', button_style="", tooltip="", icon="", layout=table_layout, style=table_style, grid_area="header1")
    code = widgets.Button(description="Status-Code", font_weight='bold', disabled=False, button_style="", tooltip="", icon="", layout=table_layout, grid_area="header1")
    body = widgets.Button(description="Body-Content", font_weight='bold', disabled=False, button_style="", tooltip="", icon="", layout=table_layout, grid_area="header1")    
    ct = widgets.Button(description="Content-Type", font_weight='bold', disabled=False, button_style="", tooltip="", icon="", layout=table_layout, grid_area="header1")
    xcto = widgets.Button(description="X-Content-Type-Options", font_weight='bold', disabled=False, button_style="", tooltip="", icon="", layout=table_layout, grid_area="header1")
    xfo = widgets.Button(description="X-Frame-Options", font_weight='bold', disabled=False, button_style="", tooltip="", icon="", layout=table_layout, grid_area="header1")
    cd = widgets.Button(description="Content-Disposition", font_weight='bold', disabled=False, button_style="", tooltip="", icon="", layout=table_layout, grid_area="header1")
    loc = widgets.Button(description="Location", disabled=False, font_weight='bold', button_style="", tooltip="", icon="", layout=table_layout, grid_area="header1")
    coop = widgets.Button(description="Cross-Origin-Opener-Policy", font_weight='bold', disabled=False, button_style="", tooltip="", icon="", layout=table_layout, grid_area="header1")
    corp = widgets.Button(description="Cross-Origin-Resource-Policy", font_weight='bold', disabled=False, button_style="", tooltip="", icon="", layout=table_layout, grid_area="header1")

    return [resp, code, body, ct, xcto, xfo, cd, loc, coop, corp]

def create_responses(title):    
    resp_title = widgets.Button(description=title, font_weight='bold', disabled=False, button_style="", tooltip="", icon="", layout=table_layout, style=table_style, grid_area=title)
    code = widgets.Dropdown(options=codes, value=200, layout=table_layout, style=table_style, grid_area=title)
    body = widgets.Dropdown(options=bodies, value="empty", layout=table_layout, style=table_style, grid_area=title)
    ct = widgets.Dropdown(options=ct_ops, value="empty", layout=table_layout, style=table_style, grid_area=title)
    xcto = widgets.Dropdown(options=xcto_ops, value="empty", layout=table_layout, style=table_style, grid_area=title)
    xfo = widgets.Dropdown(options=xfo_ops, value="empty", layout=table_layout, style=table_style, grid_area=title)
    cd = widgets.Dropdown(options=cd_ops, value="empty", layout=table_layout, style=table_style, grid_area=title)
    loc = widgets.Dropdown(options=loc_ops, value="empty", layout=table_layout, style=table_style, grid_area=title)
    coop = widgets.Dropdown(options=coop_ops, value="empty", layout=table_layout, style=table_style, grid_area=title)
    corp = widgets.Dropdown(options=corp_ops, value="empty", layout=table_layout, style=table_style, grid_area=title)
    
    resp_list = [resp_title, code, body, ct, xcto, xfo, cd, loc, coop, corp]
    resp = {"code": code, "body": body, "ct": ct, "xcto": xcto, "xfo": xfo, "cd": cd, "loc": loc, "coop": coop, "corp": corp}
    return resp, resp_list

header_list = create_header()
resp_dict["one"], resp1_list = create_responses("Response1")
resp_dict["two"], resp2_list = create_responses("Response2")
horizontal = '''
            "header header header header header header header header header header"
            "Response1 Response1 Response1 Response1 Response1 Response1 Response1 Response1 Response1 Response1"
            "Response2 Response2 Response2 Response2 Response2 Response2 Response2 Response2 Response2 Response2"
'''
vertical = '''
            "header1 Response1 Response2"
            "header1 Response1 Response2"
            "header1 Response1 Response2"
            "header1 Response1 Response2"
            "header1 Response1 Response2"
            "header1 Response1 Response2"
            "header1 Response1 Response2"
            "header1 Response1 Response2"
            "header1 Response1 Response2"
            "header1 Response1 Response2"

'''
grid = header_list + resp1_list + resp2_list
grid = zip(header_list, resp1_list, resp2_list)
grid = [y for x in grid for y in x]
#conf = widgets.GridBox(grid, layout=widgets.Layout(overflow_y="auto", grid_template_rows="auto auto auto auto auto auto auto auto auto auto", grid_template_columns=f"auto auto auto"),
#                      grid_template_areas=vertical)
conf = widgets.GridBox(grid, layout=widgets.Layout(width="60%", grid_template_rows="auto auto auto auto auto auto auto auto auto auto", grid_template_columns=f"50% 25% 25%"),
                      grid_template_areas=vertical)
display(conf)
btn = widgets.Button(description='Distinguish!')
btn.on_click(functools.partial(btn_eventhandler, resp_dict=resp_dict))
display(btn)
display(output)

In [ ]:
af = pd.DataFrame([{"req_url": "a", "resp_code": "", "cookies": True, "resp_headers": {"cross-origin-resource-policy": "same-origin"}, "resp_body_info": ":,", "resp_body_hash": ""}, {"req_url": "a", "resp_code": "", "cookies": False, "resp_headers": {}, "resp_body_info": ":,", "resp_body_hash": ""}])
display(af)
af = af.apply(fit_data, axis=1).to_dict()    
af = pd.DataFrame.from_dict(af, orient="index")
af

In [ ]:
leaky_endpoints = predict_trees(af, log=True)

In [ ]:
for method in leaky_endpoints:
    display(leaky_endpoints[method])